In [1]:
import numpy as np
import tensorflow as tf
from scipy.io import loadmat

In [2]:
num_joints = 14
batch_size = 646

In [3]:
# guassian generation
def getGaussianMap(joint = (16, 16), heat_size = 128, sigma = 2):
    # by default, the function returns a gaussian map with range [0, 1] of typr float32
    heatmap = np.zeros((heat_size, heat_size),dtype=np.float32)
    tmp_size = sigma * 3
    ul = [int(joint[0] - tmp_size), int(joint[1] - tmp_size)]
    br = [int(joint[0] + tmp_size + 1), int(joint[1] + tmp_size + 1)]
    size = 2 * tmp_size + 1
    x = np.arange(0, size, 1, np.float32)
    y = x[:, np.newaxis]
    x0 = y0 = size // 2
    g = np.exp(-((x - x0) ** 2 + (y - y0) ** 2) / (2 * (sigma ** 2)))
    g.shape
    # usable gaussian range
    g_x = max(0, -ul[0]), min(br[0], heat_size) - ul[0]
    g_y = max(0, -ul[1]), min(br[1], heat_size) - ul[1]
    # image range
    img_x = max(0, ul[0]), min(br[0], heat_size)
    img_y = max(0, ul[1]), min(br[1], heat_size)
    heatmap[img_y[0]:img_y[1], img_x[0]:img_x[1]] = g[g_y[0]:g_y[1], g_x[0]:g_x[1]]
    return heatmap

In [4]:
# read annotations
annotations = loadmat("joints.mat")
label = annotations["joints"].swapaxes(0, 2)    # shape (3, 14, 2000) -> (2000, 14, 3)

In [8]:
# label[0, :, 0] = (256 / img_shape[1])
label[1997]

array([[120.12745241, 122.67616262,   0.        ],
       [136.33061847,  98.37141354,   0.        ],
       [105.46744503, 101.26483605,   0.        ],
       [ 97.55875683,  96.63536004,   0.        ],
       [ 72.86821808, 110.90957775,   0.        ],
       [ 37.7613583 , 120.55431945,   0.        ],
       [ 75.95453543,  10.79715891,   0.        ],
       [ 91.38612215,  32.78716999,   0.        ],
       [ 97.75165167,  55.93455006,   0.        ],
       [107.5892882 ,  50.72638955,   0.        ],
       [115.88376606,  46.67559803,   0.        ],
       [119.54876791,  38.95980467,   0.        ],
       [104.11718119,  48.79744121,   0.        ],
       [ 93.70086015,  34.71611833,   0.        ]])

In [ ]:
data = np.zeros([2000, 256, 256, 3])
heatmap_set = np.zeros((2000, 128, 128, num_joints), dtype=np.float32)
print("Reading dataset...")
for i in range(2000):
    FileName = "./images/im%04d.jpg" % (i + 1)
    img = tf.io.read_file(FileName)
    img = tf.image.decode_image(img)
    img_shape = img.shape
    label[i, :, 0] *= (256 / img_shape[1])
    label[i, :, 1] *= (256 / img_shape[0])    
    data[i] = tf.image.resize(img, [256, 256])
    # generate heatmap set
    for j in range(num_joints):
        _joint = (label[i, j, 0:2] // 2).astype(np.uint16)
        heatmap_set[i, :, :, j] = getGaussianMap(joint = _joint, heat_size = 128, sigma = 4)
    # print status
    if not i%(2000//80):
        print(">", end='')